In [1]:
import os
import time
from os.path import join

import numpy as np
import torch
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter

import attack as wfpattack
from argparser import parse_taskname, trainparser
from dataset import TraceDataset
import joblib

# prase arguments
# args = trainparser().parse_args()

random_seed = 11
log_root = "./run"
ds_root = "./data"
dump_root = "./data/dump"
cache_root = "./data/cache"
time_str = time.strftime("%Y%m%d", time.localtime())
taskname = "deepexplainer-new"
log_dir = join(log_root, "shap", "RF", taskname)
dump_dir = join(dump_root, "shap", "RF", taskname)


attack: wfpattack.Attack = wfpattack.get_attack("RF")(0, 100, 0)

data = np.load("data/shap/deepexplainer-new.npz")
train_features = torch.tensor(data["features_train"])
train_labels = torch.tensor(data["labels_train"])
valid_features = torch.tensor(data["features_test"])
valid_labels = torch.tensor(data["labels_test"])


attack.init_model()

writer = SummaryWriter(log_dir)
attack.train(
    train_features,
    train_labels,
    valid_features,
    valid_labels,
    writer=writer,
    save_root=dump_dir,
)

training: 100%|██████████| 300/300 [13:53<00:00,  2.78s/it]


In [ ]:
# feature_ablation
inputxgradient
deeplift
# saliency
gradientshap